In [ ]:
!pip install mplfinance

In [ ]:
!curl -L http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O && tar xzvf ta-lib-0.4.0-src.tar.gz
!cd ta-lib && ./configure --prefix=/usr && make && make install && cd - && pip install ta-lib

In [140]:
import numpy as np
import plotly.graph_objs as go
import talib as ta
import datetime as dt
import pandas as pd

In [141]:
import pandas_datareader.data as pdr
def get_stock_data(code) :
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [144]:
df = get_stock_data(7203)

In [145]:
close =df["Close"]
df["ma5"] = ta.SMA(close, timeperiod=5)
df['ma25'] = ta.SMA(close, timeperiod=25)
df["ma75"] = ta.SMA(close, timeperiod =75)
df['ma100'] = ta.SMA(close, timeperiod=100)
df["ma200"] = ta.SMA(df["Close"], timeperiod=200)

#Golden Cross OR Dead Cross (ma5 & ma25)
cross = df["ma5"] > df["ma25"]
df["cross"] = cross
cross_shift = cross.shift(1)
temp_gc = (cross != cross_shift) & (cross ==True)
temp_dc = (cross != cross_shift) & (cross == False)
gc = [m if g == True else np.nan for g, m in zip(temp_gc, df['ma5'])]
dc = [m if d == True else np.nan for d, m in zip(temp_dc, df['ma25'])]
df["gc"], df["dc"] = gc, dc

In [146]:
#ゴールデンクロス後の上昇率
gcCount = 0
upCount = 0
maxChangeRate = 0
for gc_date in df[df["gc"].notna()].index :
  for dc_date in df[df["dc"].notna()].index:
    if gc_date < dc_date :
      gc_Close = df[df.index == gc_date]["Close"].values
      matching_rows =  df[(df.index>= gc_date) & (df.index<= dc_date)]

      if not matching_rows.empty :
        max_Close = matching_rows["Close"].max()
        changeRate = ((max_Close / gc_Close) -1)*100

        gcCount += 1

        if changeRate != 0 :
          upCount += 1
          maxChangeRate = max(maxChangeRate, changeRate)
        print(gc_date, dc_date, gc_Close, max_Close, changeRate)
      break

upRate = (upCount / gcCount) *100
print(f"UpRate: {upRate} %")
print(f"MaxChangeRate: {maxChangeRate} %")

2019-03-22 00:00:00 2019-05-09 00:00:00 [1286.42] 1356.21 [5.42513332]
2019-06-10 00:00:00 2019-08-06 00:00:00 [1300.55] 1400.62 [7.69443697]
2019-09-02 00:00:00 2019-10-04 00:00:00 [1348.26] 1448.64 [7.44515153]
2019-10-16 00:00:00 2019-11-29 00:00:00 [1465.55] 1558.73 [6.35802259]
2019-12-18 00:00:00 2019-12-26 00:00:00 [1529.04] 1539.46 [0.68147334]
2020-01-20 00:00:00 2020-01-31 00:00:00 [1529.44] 1547.52 [1.18213202]
2020-02-07 00:00:00 2020-02-18 00:00:00 [1545.56] 1545.56 [0.]
2020-03-31 00:00:00 2020-04-03 00:00:00 [1300.2] 1300.2 [0.]
2020-04-08 00:00:00 2020-04-24 00:00:00 [1350.6] 1351.2 [0.0444247]
2020-04-30 00:00:00 2020-05-08 00:00:00 [1333.2] 1333.2 [0.]
2020-05-28 00:00:00 2020-06-26 00:00:00 [1394.6] 1433.4 [2.78215976]
2020-07-21 00:00:00 2020-07-22 00:00:00 [1355.] 1355.0 [0.]
2020-08-11 00:00:00 2020-09-07 00:00:00 [1430.] 1457.0 [1.88811189]
2020-09-18 00:00:00 2020-09-24 00:00:00 [1407.6] 1416.2 [0.61096903]
2020-09-28 00:00:00 2020-10-05 00:00:00 [1435.8] 1435.8

In [134]:
#デッドクロス後の下降率
dcCount = 0
downCount = 0
maxChangeRate = 0
for dc_date in df[df["dc"].notna()].index :
  for gc_date in df[df["gc"].notna()].index:
    if dc_date < gc_date :
      dc_Close = df[df.index == dc_date]["Close"].values
      matching_rows =  df[(df.index>= dc_date) & (df.index<= gc_date)]

      if not matching_rows.empty :
        min_Close = matching_rows["Close"].min()
        changeRate = ((min_Close / dc_Close) -1)*100

        dcCount += 1

        if changeRate != 0 :
          downCount += 1
          maxChangeRate = min(maxChangeRate, changeRate)
        print(dc_date, gc_date, dc_Close, min_Close, changeRate)
      break

downRate = (downCount / dcCount) * 100
print(f"DownRate: {downRate} %")
print(f"MaxChangeRate: {maxChangeRate} %")

2019-04-03 00:00:00 2019-09-04 00:00:00 [678.] 484.0 [-28.61356932]
2019-10-08 00:00:00 2019-10-10 00:00:00 [538.] 538.0 [0.]
2019-10-11 00:00:00 2019-12-18 00:00:00 [534.] 468.0 [-12.35955056]
2019-12-23 00:00:00 2020-02-05 00:00:00 [474.] 428.0 [-9.70464135]
2020-02-18 00:00:00 2020-03-31 00:00:00 [439.] 329.0 [-25.05694761]
2020-04-01 00:00:00 2020-04-02 00:00:00 [359.] 351.0 [-2.22841226]
2020-04-03 00:00:00 2020-04-10 00:00:00 [350.] 350.0 [0.]
2020-04-24 00:00:00 2020-05-28 00:00:00 [364.] 339.0 [-6.86813187]
2020-06-16 00:00:00 2020-08-14 00:00:00 [361.] 280.0 [-22.43767313]
2020-08-24 00:00:00 2020-08-25 00:00:00 [309.] 309.0 [0.]
2020-09-09 00:00:00 2020-11-11 00:00:00 [307.] 269.0 [-12.37785016]
2020-11-30 00:00:00 2021-01-06 00:00:00 [272.] 266.0 [-2.20588235]
2021-02-18 00:00:00 2021-03-11 00:00:00 [368.] 341.0 [-7.33695652]
2021-03-23 00:00:00 2021-03-24 00:00:00 [370.] 361.0 [-2.43243243]
2021-04-02 00:00:00 2021-05-11 00:00:00 [358.] 313.0 [-12.5698324]
2021-06-21 00:00:

In [129]:
import requests
from bs4 import BeautifulSoup

url = 'https://kabutan.jp/warning/?mode=2_9'
response = requests.get(url)
html = response.text
bsObj = BeautifulSoup(html, "html.parser")

title = bsObj.find("h1").text

# 結果を格納するリスト
result_list = []

# trタグの取得
tr_tags = bsObj.find_all('tr')

# 取得したデータをリストに追加
for tr_tag in tr_tags:
    # tdタグの取得
    td_tags = tr_tag.find_all('td')

    for td_tag in td_tags:
        # aタグの取得
        a_tags = td_tag.find_all('a')

        for a_tag in a_tags:
            # aタグのテキストをリストに追加
            result_list.append(a_tag.text)

result_list = list(filter(None, result_list))
print(result_list)

['9501', '8304', '6526', '9107', '6857', '2127', '6920', '3659', '1570', '4005', '8306', '6902', '7203', '9101', '8801']
